In [26]:
import copy
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

from sequence_jacobian import het, simple, create_model              # functions
from sequence_jacobian import interpolate, grids, misc, estimation   # modules

def bissection_onestep(f,a,b):
    if not np.all(f(a)*f(b) <= 0):
        raise ValueError("No sign change")
    else:
        mid_point = (a + b)/2
        mid_value = f(mid_point)
        new_a = a
        new_b = b
        indxs_a = np.nonzero(mid_value*f(b) <= 0)
        indxs_b = np.nonzero(mid_value*f(a) <= 0)
        if indxs_a[0].size != 0:
            new_a[indxs_a] = mid_point[indxs_a]
        if indxs_b[0].size != 0:
            new_b[indxs_b] = mid_point[indxs_b]
        return new_a,new_b

def vec_bissection(f,a,b,iter_max = 100,tol = 1E-11):
    i = 1
    err = 1
    while i < iter_max and err > tol:
        a,b = bissection_onestep(f,a,b)
        err = np.max(np.abs(a - b))
        i += 1
    if i >= iter_max:
        raise Error("No convergence")
    return a

def consumption(c,we,rest,gamma,v,phi):
    return c - we*(we/(phi*c**gamma))**(1/v) - rest

def household_guess(a_grid,e_grid,r,w,gamma,T):
    wel = (1+r)*a_grid[np.newaxis,:] + w*e_grid[:,np.newaxis] + T[:,np.newaxis]
    V_prime = (1+r)*(wel*0.1)**(-gamma)
    return V_prime

@het(exogenous = 'Pi',policy = 'a', backward = 'V_prime', backward_init=household_guess)
def household(V_prime_p,a_grid,e_grid,r,N,w,T,beta,gamma,v,phi):

    we = w*e_grid

    c_prime = (beta*V_prime_p)**(-1/gamma) #c_prime is quite a misnomer, since this is the new guess for c_t

    new_grid = (c_prime + a_grid[np.newaxis,:] - we[:,np.newaxis]*N - T[:,np.newaxis])
    wel = (1+r)*a_grid

    c = interpolate.interpolate_y(new_grid,wel,c_prime)

    a = wel + we[:,np.newaxis]*N + T[:,np.newaxis] - c
    V_prime= (1+r)*c**(-gamma)

    # checks for violations of the condition of minimal assets required and fixes it

    indexes_asset = np.nonzero(a < a_grid[0]) #first dimension: labor grid, second dimension: asset grid
    a[indexes_asset] = a_grid[0]

    if indexes_asset[0].size != 0 and indexes_asset[1].size !=0:
        
        c[indexes_asset] = c[0]
        
        V_prime[indexes_asset] = (1+r)*(c[indexes_asset])**(-gamma)
        
    uce = e[:,np.newaxis]*c**(-gamma)

    return V_prime,a,c,uce

print(household)
print(f'Inputs: {household.inputs}')
print(f'Macro outputs: {household.outputs}')
print(f'Micro outputs: {household.internals}')

<HetBlock 'household'>
Inputs: ['a_grid', 'e_grid', 'r', 'N', 'w', 'T', 'beta', 'gamma', 'v', 'phi', 'Pi']
Macro outputs: ['A', 'C', 'UCE']
Micro outputs: ['D', 'Dbeg', 'Pi', 'V_prime', 'a', 'c', 'uce']


In [27]:
def make_grid(rho_e, sd_e, nE, amin, amax, nA):
    e_grid, pi_e, Pi = grids.markov_rouwenhorst(rho=rho_e, sigma=sd_e, N=nE)
    a_grid = grids.agrid(amin=amin, amax=amax, n=nA)
    return e_grid, Pi, a_grid, pi_e

def transfers(pi_e, Div, Transfer, e_grid):
    # hardwired incidence rules are proportional to skill; scale does not matter 
    tax_rule, div_rule = np.ones(e_grid.size), e_grid #np.ones(e_grid.size)
    div = Div / np.sum(pi_e * div_rule) * div_rule
    transfer =  (Transfer) / np.sum(pi_e * tax_rule) * tax_rule 
    T = div + transfer
    return T

hh_inp = household.add_hetinputs([make_grid,transfers])

print(hh_inp)
print(f'Inputs: {hh_inp.inputs}')
print(f'Outputs: {hh_inp.outputs}')

<HetBlock 'household' with hetinput 'make_grid_transfers'>
Inputs: ['r', 'N', 'w', 'beta', 'gamma', 'v', 'phi', 'rho_e', 'sd_e', 'nE', 'amin', 'amax', 'nA', 'Div', 'Transfer']
Outputs: ['A', 'C', 'UCE']


In [30]:
@simple
def firm(Y, w, Z, pi, mu, kappa):
    L = Y / Z
    Div = Y - w * L - mu/(mu-1)/(2*kappa) * (1+pi).apply(np.log)**2 * Y
    return L, Div


@simple
def monetary(pi, rstar, phi_pi):
    r = (1 + rstar(-1) + phi_pi * pi(-1)) / (1 + pi) - 1
    return r


@simple
def fiscal(r,Transfer,B):
    govt_res = (1+r)*B(-1) + Transfer - B
    return govt_res

@simple
def mkt_clearing(A, N, C, L, Y, B, pi, mu, kappa):
    asset_mkt = A - B
    goods_mkt = Y - C - mu/(mu-1)/(2*kappa) * (1+pi).apply(np.log)**2 * Y
    return asset_mkt, labor_mkt, goods_mkt


@simple
def nkpc_ss(Z, mu):
    w = Z / mu
    return w

@simple
def wage(pi, w):
    piw = (1 + pi) * w / w(-1) - 1
    return piw

@simple
def union(piw,w,N,uce,kw,phi,v,muw,beta):
   wnkpc = kw*(phi*N**(1+v) - w*N/muw*uce) + beta*(1 + piw(+1)).apply(np.log) - (1 + piw).apply(np.log)
   return wnkpc

@simple 
def union_ss(w,N,UCE,kw,v,muw):
    phi = (w*N*UCE)/(muw*N**(1+v))
    wnkpc = kw*(phi*N**(1+v)-w*N/muw*UCE)
    return wnkpc, phi

In [31]:
blocks_ss = [hh_inp, firm, monetary, fiscal, mkt_clearing, nkpc_ss, union_ss]

hank_ss = create_model(blocks_ss, name="One-Asset HANK SS")

print(hank_ss)
print(f"Inputs: {hank_ss.inputs}")

Exception: Topological sort failed: cyclic dependency household -> union_ss -> household

In [24]:
calibration = {'gamma': 1, 'v':1, 'rho_e': 0.966, 'sd_e': 0.5, 'nE': 7,
               'amin': 0, 'amax': 150, 'nA': 500, 'Y': 1.0, 'Z': 1.0, 'pi': 0.0,
               'mu': 1.2, 'kappa': 0.1, 'rstar': 0.005, 'phi_pi': 0,'B':5.6,
                'kw': 0.1, 'muw': 1.1, 'N': 1.0}

unknowns_ss = {'beta': 0.986, 'phi': 0.8, 'Transfer': -0.03}
targets_ss = {'asset_mkt': 0, 'labor_mkt': 0, 'govt_res': 0}

ss0 = hank_ss.solve_steady_state(calibration, unknowns_ss, targets_ss, solver="hybr")

KeyError: 'uce'

In [ ]:
print(f"Asset market clearing: {ss0['asset_mkt']: 0.2e}")
print(f"Labor market clearing: {ss0['labor_mkt']: 0.2e}")
print(f"Govt Budget Constrain: {ss0['govt_res']: 0.2e}")
print(f"Goods market clearing (untargeted): {ss0['goods_mkt']: 0.2e}")